In [1]:
!pip install torch torchvision pillow scikit-learn tqdm

   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   -------- ------------------------------- 1.3/6.2 MB 4.5 MB/s eta 0:00:02
   --------------- ------------------------ 2.4/6.2 MB 5.0 MB/s eta 0:00:01
   ---------------------- ----------------- 3.4/6.2 MB 5.0 MB/s eta 0:00:01
   ---------------------------- ----------- 4.5/6.2 MB 5.1 MB/s eta 0:00:01
   ------------------------------------- -- 5.8/6.2 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 5.3 MB/s  0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0


In [1]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
from pathlib import Path


In [2]:
DATASET_PATH = r"C:\Users\Student\Downloads\archive (21)\Crop Diseases Dataset" # <-- your actual dataset folder


In [3]:
IMG_SIZE = 384
BATCH_SIZE = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.1)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_tfms)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

val_ds.dataset.transform = val_tfms

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

classes = full_dataset.classes
print("Total Classes:", len(classes))
print("Example classes:", classes[:10])


Total Classes: 17
Example classes: ['Bacterial Blight', 'Corn___Common_Rust', 'Corn___Gray_Leaf_Spot', 'Corn___Healthy', 'Corn___Northern_Leaf_Blight', 'Healthy', 'Potato___Early_Blight', 'Potato___Healthy', 'Potato___Late_Blight', 'Red Rot']


In [4]:
def build_model(num_classes):
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = build_model(len(classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

best_acc = 0.0
EPOCHS = 100

for epoch in range(EPOCHS):
    model.train()
    train_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        correct += (out.argmax(1) == labels).sum().item()
        total += labels.size(0)

    scheduler.step()
    train_acc = correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_dl:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            val_correct += (out.argmax(1) == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")

    # Save best
    # if val_acc > best_acc:
    #     best_acc = val_acc
    torch.save({
        "state_dict": model.state_dict(),
        "classes": classes
    }, "crop_disease_model.pt")
    print("✅ Best model saved (val_acc=", round(best_acc, 3), ")")


Using device: cuda


Epoch 1/100: 100%|█████████████████████████████████████████████████████████████████| 1333/1333 [05:50<00:00,  3.80it/s]


Epoch 1: Train Acc=0.872, Val Acc=0.940
✅ Best model saved (val_acc= 0.0 )


Epoch 2/100: 100%|█████████████████████████████████████████████████████████████████| 1333/1333 [05:47<00:00,  3.83it/s]


Epoch 2: Train Acc=0.940, Val Acc=0.928
✅ Best model saved (val_acc= 0.0 )


Epoch 3/100:  19%|████████████▌                                                     | 254/1333 [01:06<05:37,  3.20it/s]

In [7]:
pesticides = {
    "Corn___Common_Rust": ["Apply Mancozeb or Propiconazole", "Use resistant varieties."],
    "Corn___Gray_Leaf_Spot": ["Use Strobilurin fungicides", "Rotate crops to reduce disease pressure."],
    "Corn___Northern_Leaf_Blight": ["Use Azoxystrobin + Propiconazole", "Maintain good air circulation."],
    "Potato___Early_Blight": ["Use Chlorothalonil or Mancozeb", "Avoid overhead irrigation."],
    "Potato___Late_Blight": ["Use Metalaxyl or Cymoxanil-based sprays", "Remove infected leaves."],
    "Rice___Leaf_Blast": ["Spray Tricyclazole or Isoprothiolane", "Avoid excessive nitrogen."],
    "Rice___Brown_Spot": ["Use Mancozeb or Thiophanate-methyl", "Apply balanced fertilizer."],
    "Rice___Neck_Blast": ["Apply Tricyclazole", "Maintain proper spacing."],
    "Wheat___Yellow_Rust": ["Spray Tebuconazole", "Use resistant varieties."],
    "Wheat___Brown_Rust": ["Apply Propiconazole", "Remove volunteer plants."],
    "sugarcane/Bacterial Blight": ["Use Copper oxychloride", "Destroy infected clumps."],
    "sugarcane/Red Rot": ["Use Carbendazim", "Avoid ratooning diseased crops."],
    "Healthy": ["No pesticide needed. Maintain good field hygiene."]
}

with open("pesticides.json", "w") as f:
    json.dump(pesticides, f, indent=2)
print("✅ Pesticide map saved.")


✅ Pesticide map saved.


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

def load_model(model_path="crop_disease_model.pt"):
    ckpt = torch.load(model_path, map_location=device)
    model = build_model(len(ckpt["classes"]))
    model.load_state_dict(ckpt["state_dict"])
    model = model.to(device)  # <-- Important fix: send model to device
    model.eval()
    return model, ckpt["classes"]

def predict_image(img_path, model, classes):
    tfms = transforms.Compose([
        transforms.Resize(384),
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    img = Image.open(img_path).convert("RGB")
    x = tfms(img).unsqueeze(0).to(device)  # input to same device
    model = model.to(device)               # model to same device

    with torch.no_grad():
        out = model(x)
        probs = torch.softmax(out, dim=1)
        top_idx = probs.argmax(dim=1).item()
        conf = probs[0, top_idx].item()

    return classes[top_idx], conf


✅ Using device: cuda


In [15]:
model, classes = load_model("crop_disease_model.pt")

img_path = r"C:\Users\Student\Downloads\archive (21)\Crop Diseases Dataset\Rice___Leaf_Blast\rb_wb_3.jpg"

pred_class, conf = predict_image(img_path, model, classes)
print(f"Prediction: {pred_class} ({conf*100:.2f}% confidence)")


C:\Users\Student\AppData\Local\Temp\ipykernel_7132\3902348480.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location=device)


Prediction: Potato (100.00% confidence)


In [16]:
with open("pesticides.json", "r") as f:
    pesticides = json.load(f)

recommendations = pesticides.get(pred_class, ["No data available"])
print("\nRecommended Actions:")
for rec in recommendations:
    print("-", rec)



Recommended Actions:
- No data available
